### 1. Install required packages

In [ ]:
!pip install -r ../requirements.txt

### 2. Connect to google drive
My google drive directory will be mounted locally on `~/gdrive` folder

In [ ]:
from google.colab import drive
drive.mount('gdrive')

### 3. Clone the project repository from github
This repos contains utility functions, for data processing, model definition and model training

In [ ]:
!git clone https://github.com/AlkaSaliss/DEmoClassi

### 4. Download data from kaggle
To use the kaggle API we need to download a google credentials from our kaggle account and save it to google drive so that we can access to it from google colab.

After connecting to kaggle we'll download the fer2013 face image datasets used for facial expression classification.

In [ ]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

In [ ]:
# download the data
if not os.path.exists('~/.kaggle'):
    os.system('mkdir ~/.kaggle')

!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

In [ ]:
# create the data directory if not exist
if not os.path.exists('/content/data'):
    os.mkdir('/content/data')

In [ ]:
# extract ferg2013 dataset in the /content/data/ directory
!tar -zxvf /content/fer2013.tar.gz -C /content/data/

In [ ]:
# install cmake in order to compile some modules from source
!apt-get -y install cmake

### 1. Install needed packages

In [ ]:
%%time

# Installing the gpu version of dlib package
! git clone https://github.com/davisking/dlib.git
%cd dlib
! mkdir build
%cd build
! cmake .. -DDLIB_USE_CUDA=1 -DUSE_AVX_INSTRUCTIONS=1
! cmake --build .
%cd ..
! python setup.py install --yes USE_AVX_INSTRUCTIONS --yes DLIB_USE_CUDA
%cd /content

In [ ]:
# installing imutils and pytorch, kaggle
!pip install --upgrade imutils
!pip install torchvision
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu90/torch_nightly.html
!pip install kaggle
#!pip install face_recognition

### 2. Downloading the datasets

Connecting to kaggle to donwload ferg2013 dataset

Connect to drive to download jaffedbase dataset

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

### 3. Image processing

In [ ]:
# package import 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import Image
from scipy import misc
from scipy.misc.pilutil import imread, imresize, imsave
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
import torch.nn.functional as F
import torch.optim as optim
import os
import shutil
import gzip
import glob
import random
from random import shuffle
import tqdm
import cv2
from skimage import transform
from skimage.color import rgb2gray
from skimage import exposure
import skimage
import warnings
warnings.filterwarnings('ignore')
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
import h5py
import time
import copy
import imutils
import dlib
from imutils.face_utils import FaceAligner, rect_to_bb
from multiprocessing import Pool
from collections import Counter
import itertools

In [ ]:
# !rm -r /content/dlib/

In [ ]:
# import face_recognition

In [ ]:
print(torch.__version__)

Now let's extract the dataset into data directory

#### 3.1 Loading and processing ferg2013 data


In [ ]:
fer2013 = pd.read_csv('/content/data/fer2013/fer2013.csv')
# print some information
fer2013.info(memory_usage='deep')

In [ ]:
# print some information
fer2013.info(memory_usage='deep')

In [ ]:
# print some lines
fer2013.sample(10)

In [ ]:
# print the count for train - public - private sets
fer2013.Usage.value_counts()

In [ ]:
# print counts for facial expressions
fer2013.emotion.value_counts()

In [ ]:
# check if there's missoing values
fer2013.isnull().sum()

In [ ]:
# reshape the images to get 48x48 pixels images
fer2013_array = fer2013.pixels.values
fer2013_array = np.array([[int(pix) for pix in img.split()] 
                          for img in fer2013_array])
fer2013_array = fer2013_array.reshape((fer2013_array.shape[0], 48, 48))

In [ ]:
print('Number of samples : {}'.format(fer2013_array.shape[0]))
print("Images shape : {}x{} ".format(fer2013_array.shape[1], fer2013_array.shape[2]))

In [ ]:
# get the labels
fer2013_labels = fer2013.emotion.values
# get the flag (train, private and public set)
flags = fer2013.Usage.values

plot some sample images for each expression

In [ ]:
def plot_examples(label):
    dict_label = {
        0: 'Angry',
        1: 'Disgust',
        2: 'Fear',
        3: 'Happy',
        4: 'Sad',
        5: 'Surprise',
        6: 'Neutral'
    }
    
    print('Images belonging to class:', dict_label[label])
    inds1 = set(np.where(fer2013_labels==label)[0])
    inds2 = set(np.where(flags=='Training')[0])
    inds = list(inds1.intersection(inds2))
    sample_inds = np.random.choice(inds, 9, replace=False)
    
    fig, ax = plt.subplots(nrows=3,ncols=3,figsize=(20,10))
    ax = ax.ravel()
    for idx, e in enumerate(sample_inds):
        img = fer2013_array[e]
        ax[idx].imshow(img, cmap='gray')

In [ ]:
plot_examples(0)

In [ ]:
plot_examples(1)

In [ ]:
plot_examples(2)

In [ ]:
plot_examples(3)

In [ ]:
plot_examples(4)

In [ ]:
plot_examples(5)

In [ ]:
plot_examples(6)

#### 3.2 Loading and processing jaffe data

Each jaffe image has its label incorporated in its file name.
Let's load the file names and explore the different expressions

In [ ]:
paths_jaffe = sorted(glob.glob('/content/data/jaffe/*.tiff'))

In [ ]:
print('Number of images: {}'.format(len(paths_jaffe)))

In [ ]:
# get the labels
jaffe_labels = np.array([im.split('.')[1][:2] for im in paths_jaffe])

In [ ]:
print('frequency for each expression')
unique, counts = np.unique(jaffe_labels, return_counts=True)
print(np.asarray((unique, counts)).T)

Let's create a function that read a tiff file and returns its numpy representation along with its label.

jaffe image label is incorporated in the file name

In [ ]:
def load_tiff_image(path):
    dict_label = {
        0: 'AN',
        1: 'DI',
        2: 'FE',
        3: 'HA',
        4: 'SA',
        5: 'SU',
        6: 'NE'
    }
    rev_dict = dict([(v, k) for k, v in dict_label.items()])
    
    im = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    label = rev_dict[path.split('.')[1][:2]]
    
    return (im, label)

In [ ]:
jaffe_data = [load_tiff_image(p) for p in tqdm.tqdm(paths_jaffe)]
jaffe_array = np.concatenate([np.expand_dims(item[0], 0) for item in jaffe_data])
jaffe_labels = np.array([item[1] for item in jaffe_data])

In [ ]:
assert len(jaffe_array) == len(jaffe_labels)
print('jaffe data shape', jaffe_array.shape, jaffe_labels.shape)

In [ ]:
def plot_jaffe(label):
    dict_label = {
        0: 'Angry',
        1: 'Disgust',
        2: 'Fear',
        3: 'Happy',
        4: 'Sad',
        5: 'Surprise',
        6: 'Neutral'
    }
    
    print('Images belonging to class:', dict_label[label])
    inds = np.where(jaffe_labels==label)[0]
    sample_inds = np.random.choice(inds, 9, replace=False)
    
    fig, ax = plt.subplots(nrows=3,ncols=3,figsize=(20,10))
    ax = ax.ravel()
    for idx, e in enumerate(sample_inds):
        img = jaffe_array[e]
        ax[idx].imshow(img, cmap='gray')

In [ ]:
plot_jaffe(0)

In [ ]:
plot_jaffe(1)

In [ ]:
plot_jaffe(2)

In [ ]:
plot_jaffe(3)

In [ ]:
plot_jaffe(4)

In [ ]:
plot_jaffe(5)

In [ ]:
plot_jaffe(6)

#### 3.3 Further processing steps:
* face alignement
* image cropping
* image resizing

Define a function that applies all these prcessing steps to an image

In [ ]:
# download the pretrained face_detector file for dlib
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 -P /content/data

In [ ]:
!bzip2 -dk /content/data/shape_predictor_68_face_landmarks.dat.bz2

In [ ]:
PATH_DETECTOR = '/content/data/shape_predictor_68_face_landmarks.dat'
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(PATH_DETECTOR)

In [ ]:
# function to align faces and compute landmarks
def align_and_crop(list_images, desiredFaceWidth=256, convert=True, resize=(800, 800)):
    
    # converting image to 8-bit
    if convert:
        print('--------------Converting images----------------')
        list_images = list(np.uint8(list_images))
    #resizing the  images
    if resize:
        print('--------------Resizing images----------------')
        #list_images = [transform.resize(np.repeat(np.expand_dims(im, 2), 3, 2), resize).astype('float32')
        #               for im in list_images]
        list_images = [np.repeat(np.expand_dims(im, 2), 3, 2) 
                       for im in list_images]
        
    # list_images = list(list_images)
    

    
    results = []
    print('----------------- Face alignment-------------------')
    for im in list_images:
        gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
        # detect the face in the image
        rects = detector(gray, 2)
        if len(rects) > 0: # if we have at least one face detected
            rects = rects[0] # consider the 1st face
            faceAligned = fa.align(im, gray, rects) # align the face
            
            # cropping the image
            gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
            rects = detector(faceAligned, 2)
            if len(rects) > 0:
                rects = rects[0]
                (x, y, w, h) = rect_to_bb(rects) # get the bounding box coordinates
                faceCropped = transform.resize(faceAligned[y: y + h, 
                                                     x: x + w, 0], (48, 48))
                # shape = imutils.face_utils.shape_to_np(predictor(faceAligned, rects))
                # compute the cropping coordinates
                # center = shape[27]
                # right_eye = np.mean(shape[42:48], axis=0).astype(int)
                # alpha = right_eye[0] - center[0]
                # new_x = np.floor(center[0] - 1.2 * alpha).astype(int)
                # new_y = np.floor(center[1] - 1.3 * alpha).astype(int)
                # new_w = np.ceil(2.4 * alpha).astype(int)
                # new_h = np.ceil(4.5 * alpha).astype(int)
                  
                # crop the face and resize image to 48x48
                # faceCropped = transform.resize(faceAligned[new_y: new_y + new_h, 
                #                                     new_x: new_x + new_w, 0], (48, 48))
                print('OK 1')
                results.append(faceCropped)
            else:
                print('OK 2')
                # if no face is found in the aligned face, return just a resized aligned face 48x48
                # results.append(transform.resize(faceAligned[:, :, 0], (48, 48)))
                results.append(faceAligned[:, :, 0])
        else:
            print('OK 3')
            # if no face is found at all in the image, return the resized original image 48x48
            # results.append(transform.resize(im[:, :, 0], (48, 48)))
            results.append(im[:, :, 0])
    
    # apply histogram equalization to all images and return the result
    return np.concatenate(
        [np.expand_dims(exposure.equalize_hist(item), 0) for item in results]
    )


In [ ]:
# Instanciate a face aligner object
fa = FaceAligner(predictor, desiredFaceWidth=48)

# function to process one image
def align_and_crop_one(im):

    gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
    # detect the face in the image
    rects = detector(gray, 2)
    if len(rects) > 0: # if we have at least one face detected
        rects = rects[0] # consider the 1st face
        faceAligned = fa.align(im, gray, rects) # align the face

        # cropping the image
        gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
        rects = detector(faceAligned, 2)
        if len(rects) > 0:
            rects = rects[0]
            (x, y, w, h) = rect_to_bb(rects) # get the bounding box coordinates
            faceCropped = transform.resize(faceAligned[y: y + h, 
                                                 x: x + w, 0], (48, 48))

            # print('OK 1')
            return (exposure.equalize_hist(faceCropped), 'OK1')
        else:
            # print('OK 2')
            # if no face is found in the aligned face, return just a resized aligned face 48x48
            return (exposure.equalize_hist(faceAligned[:, :, 0]), 'OK2')
    else:
        # print('OK 3')
        # if no face is found at all in the image, return the resized original image 48x48
        return (exposure.equalize_hist(transform.resize(im[:, :, 0], (48, 48))), 'OK3')


# function to align faces and compute landmarks
def align_and_crop(list_images, convert=True, reshape=True):
    # converting image to 8-bit
    if convert:
        print('--------------Converting images----------------')
        list_images = list(np.uint8(list_images))
    #resizing the  images
    if reshape:
        print('--------------Resizing images----------------')
        list_images = [np.repeat(np.expand_dims(im, 2), 3, 2) 
                       for im in list_images]
    
    with Pool() as p:
        results_tuple = p.map(align_and_crop_one, tqdm.tqdm(list_images))
    
    results1 = [item[0] for item in results_tuple]
    results2 = [item[1] for item in results_tuple]
    
    # apply histogram equalization to all images and return the result
    return np.concatenate([np.expand_dims(item, 0) for item in results1]), dict(Counter(results2))


In [ ]:
%%time
fer2013_transformed, counts = align_and_crop(fer2013_array)

In [ ]:
%%time
jaffe_transformed, counts_j = align_and_crop(jaffe_array)

Save the data to gdrive

In [ ]:
#path to Google Drive
PATH_DRIVE = '/content/gdrive/My Drive/Face_detection'


In [ ]:
%%time

np.save(os.path.join(PATH_DRIVE, 'fer2013_transformed.npy'), fer2013_transformed)
np.save(os.path.join(PATH_DRIVE, 'fer2013_labels.npy'), fer2013_labels)
np.save(os.path.join(PATH_DRIVE, 'fer2013_array.npy'), fer2013_array)
np.save(os.path.join(PATH_DRIVE, 'flags.npy'), flags)

np.save(os.path.join(PATH_DRIVE, 'jaffe_array.npy'), jaffe_array)
np.save(os.path.join(PATH_DRIVE, 'jaffe_labels.npy'), jaffe_labels)
np.save(os.path.join(PATH_DRIVE, 'jaffe_transformed.npy'), jaffe_transformed)

### 4. Modeling phase

In [ ]:
%%time

# Load the data from drive
PATH_DRIVE = '/content/gdrive/My Drive/Face_detection'

fer2013_transformed = np.load(os.path.join(PATH_DRIVE, 'fer2013_transformed.npy'))
fer2013_labels = np.load(os.path.join(PATH_DRIVE, 'fer2013_labels.npy'))
fer2013_array = np.load(os.path.join(PATH_DRIVE, 'fer2013_array.npy'))
flags = np.load(os.path.join(PATH_DRIVE, 'flags.npy'))

jaffe_array = np.load(os.path.join(PATH_DRIVE, 'jaffe_array.npy'))
jaffe_labels = np.load(os.path.join(PATH_DRIVE, 'jaffe_labels.npy'))
jaffe_transformed = np.load(os.path.join(PATH_DRIVE, 'jaffe_transformed.npy'))

#### 4.1 Data preparation

Create train - validation - test sets

In [ ]:
# train set
train_fer2013_images = fer2013_transformed[np.where(flags=='Training')[0]]
train_fer2013_labels = fer2013_labels[np.where(flags=='Training')[0]]

# validation set
val_fer2013_images = fer2013_transformed[np.where(flags=='PublicTest')[0]]
val_fer2013_labels = fer2013_labels[np.where(flags=='PublicTest')[0]]

# test set
test_fer2013_images = fer2013_transformed[np.where(flags=='PrivateTest')[0]]
test_fer2013_labels = fer2013_labels[np.where(flags=='PrivateTest')[0]]

In [ ]:
print(train_fer2013_images.shape, train_fer2013_labels.shape)
print(val_fer2013_images.shape, val_fer2013_labels.shape)
print(test_fer2013_images.shape, test_fer2013_labels.shape)

In [ ]:
def plot_images(images, labels, class_):
    dict_label = {
        0: 'Angry',
        1: 'Disgust',
        2: 'Fear',
        3: 'Happy',
        4: 'Sad',
        5: 'Surprise',
        6: 'Neutral'
    }
    
    print('Images belonging to class:', dict_label[class_])
    inds = np.where(labels==class_)[0]
    sample_inds = np.random.choice(inds, 16, replace=False)
    
    fig, ax = plt.subplots(nrows=4,ncols=4,figsize=(22,10))
    ax = ax.ravel()
    for idx, e in enumerate(sample_inds):
        img = images[e]
        ax[idx].imshow(img, cmap='gray')

In [ ]:
plot_images(jaffe_transformed, jaffe_labels, 6)

Now we concatenate fer2013 data and jaffe

In [ ]:
train_images = np.concatenate([train_fer2013_images, jaffe_transformed])
train_labels = np.concatenate([train_fer2013_labels, jaffe_labels])

In [ ]:
print(train_images.shape, train_labels.shape)
print(val_fer2013_images.shape, val_fer2013_labels.shape)
print(test_fer2013_images.shape, test_fer2013_labels.shape)

In [ ]:
# Let's create a pytorch dataset class
1+1

In [ ]:
class FaceData(Dataset):
    """Face image dataset."""
    
    def __init__(self, images, labels, transform=None):
        assert images.shape[0] == labels.shape[0]
        self.images = images
        self.labels = np.expand_dims(labels, 1)
        self.transform = transform
    
    def __len__(self):
        return self.images.shape[0]
    
    def __getitem__(self, idx):
        sample = (np.expand_dims(self.images[idx], 0), self.labels[idx])
        if self.transform:
            sample = self.transform(sample)
        return sample

# Transformer to convert numpy into pytorch tensor
class ToTensor(object):
    def __call__(self, sample):
        return (torch.from_numpy(sample[0]), torch.from_numpy(sample[1]))

In [ ]:
# Define some constants
N_EPOCHS = 25
BATCH_SIZE = 128

In [ ]:
# Instanciate a transform object
list_transforms = transforms.Compose([ToTensor()])

# Create train, test and validation data objects
face_datasets = {
    'train': FaceData(train_images, train_labels, list_transforms),
    'val': FaceData(val_fer2013_images, val_fer2013_labels, list_transforms),
}

face_datasets_test = FaceData(test_fer2013_images, test_fer2013_labels, list_transforms)

# Data loaders
dataloaders = {
    phase: DataLoader(face_datasets[phase], batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    for phase in ['train', 'val']
}
dataset_sizes = {phase: len(face_datasets[phase]) for phase in ['train', 'val']}

dataloaders_test = DataLoader(face_datasets_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
dataset_sizes_test = len(face_datasets_test)

In [ ]:
# check if train data loader is OK
tmp = next(iter(dataloaders['train']))
print(tmp[0].size())
print(tmp[1].size())

In [ ]:
# check if validation data loader is OK
tmp = next(iter(dataloaders['val']))
print(tmp[0].size())
print(tmp[1].size())

In [ ]:
# check if test data loader is OK
tmp = next(iter(dataloaders_test))
print(tmp[0].size())
print(tmp[1].size())

In [ ]:
print(tmp[0].dtype)
print(tmp[1].dtype)

#### 4.2 Create a model

We'll implement a depthwise separable convolution network

Depthwise separable conv layer

In [ ]:
class SeparableConv(torch.nn.Module):
    """Depthwise separable convolution layer implementation."""
    
    def __init__(self, nin, nout, kernel_size=3):
        super(SeparableConv, self).__init__()
        self.depthwise = torch.nn.Conv2d(nin, nin, kernel_size=kernel_size, groups=nin)
        self.pointwise = torch.nn.Conv2d(nin, nout, kernel_size=1)
    
    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

In [ ]:
class Model(torch.nn.Module):
    
    def __init__(self, dropout=0.3, n_class=7, n_filters=[64, 128, 256, 512]):
        super(Model, self).__init__()
        
        self.dropout = dropout
        self.n_class = n_class
        self.n_filters = n_filters
        
        # 1st block
        self.conv1 = SeparableConv(1, self.n_filters[0])
        self.batchnorm1 = torch.nn.BatchNorm2d(self.n_filters[0])
        self.conv2 = SeparableConv(self.n_filters[0], self.n_filters[0])
        self.batchnorm2 = torch.nn.BatchNorm2d(self.n_filters[0])
        
        # 2nd block
        self.conv3 = SeparableConv(self.n_filters[0], self.n_filters[1])
        self.batchnorm3 = torch.nn.BatchNorm2d(self.n_filters[1])
        self.conv4 = SeparableConv(self.n_filters[1], self.n_filters[1])
        self.batchnorm4 = torch.nn.BatchNorm2d(self.n_filters[1])
        
        # 3rd block
        self.conv5 = SeparableConv(self.n_filters[1], self.n_filters[2])
        self.batchnorm5 = torch.nn.BatchNorm2d(self.n_filters[2])
        self.conv6 = SeparableConv(self.n_filters[2], self.n_filters[2])
        self.batchnorm6 = torch.nn.BatchNorm2d(self.n_filters[2])
        
        # 4th block
        self.conv7 = SeparableConv(self.n_filters[2], self.n_filters[3])
        self.batchnorm7 = torch.nn.BatchNorm2d(self.n_filters[3])
        self.conv8 = SeparableConv(self.n_filters[3], self.n_filters[3])
        self.batchnorm8 = torch.nn.BatchNorm2d(self.n_filters[3])
        
        self.avg_pool = torch.nn.AdaptiveAvgPool2d((1, 1))
        
        # 1st fc block
        self.fc1 = torch.nn.Linear(self.n_filters[3], 256)
        self.batchnorm9 = torch.nn.BatchNorm1d(256)
        
         # 2nd fc block
        self.fc2 = torch.nn.Linear(256, 128)
        self.batchnorm10 = torch.nn.BatchNorm1d(128)
        
         # output block
        self.fc3 = torch.nn.Linear(128, self.n_class)
        
    def forward(self, x):
        #1st block
        x = self.conv1(x)
        x = F.relu(self.batchnorm1(x))
        x = self.conv2(x)
        x = F.relu(self.batchnorm2(x))
        x = F.max_pool2d(x, 2)
        
        # 2nd block
        x = self.conv3(x)
        x = F.relu(self.batchnorm3(x))
        x = self.conv4(x)
        x = F.relu(self.batchnorm4(x))
        x = F.max_pool2d(x, 2)
        
        # 3rd block
        x = self.conv5(x)
        x = F.relu(self.batchnorm5(x))
        x = self.conv6(x)
        x = F.relu(self.batchnorm6(x))
        # x = F.max_pool2d(x, 2)
                
        # 4th block
        x = self.conv7(x)
        x = F.relu(self.batchnorm7(x))
        x = self.conv8(x)
        x = F.relu(self.batchnorm8(x))
        # x = F.max_pool2d(x, 2)
        
        x = self.avg_pool(x)
        x = F.dropout(x.view(-1, x.size()[1]), self.dropout)
        
        x = F.relu(self.batchnorm9(self.fc1(x)))
        x = F.dropout(x, self.dropout)
        
        x = F.relu(self.batchnorm10(self.fc2(x)))
        x = F.dropout(x, self.dropout)
        
        x = self.fc3(x)
        
        return x


#### 4. 3 Training phase

In [ ]:
# check if GPU available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
# instantiate model and optimizers
model = Model()
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
print(model)

In [ ]:
# utility function to get the number of parameters in a model
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
print('Number of trainable paramaters: {}'.format(count_parameters(model)))

Let's create a training utility function

In [ ]:
def train_model(model, criterion, optimizer, data=dataloaders, dataset_sizes=dataset_sizes, num_epochs=N_EPOCHS):
    since = time.time()
    acc_history = {'train': [], 'val': []}
    loss_history = {'train': [], 'val': []}
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # set model to training mode
            else:
                model.eval()   # set model to evaluation mode
        
            running_loss = 0.0
            running_corrects = 0
            
            # Iterate over data.
            for inputs, labels in data[phase]:
                # inputs = inputs.to(device, dtype=torch.float)
                inputs = inputs.to(device, dtype=torch.float)
                labels = labels.view(labels.size()[0]).to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()
                
                # forward pass
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    
                    # loss = criterion(outputs, labels)
                    loss = criterion(outputs, labels)
                    
                    # Backward pass + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
            
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            acc_history[phase].append(epoch_acc.to('cpu').numpy().mean()) 
            loss_history[phase].append(epoch_loss) 
            
            print('{} Loss: {:.4f}, Acc: {:.4f}'.format(
            phase, epoch_loss, epoch_acc))
            
            # deep copy th emodel
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            
        print()
        
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
            time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:.4f}'.format(best_acc))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, acc_history, loss_history


In [ ]:
# helper plot function to plot the training history
def plot_train_history(accuracies, losses):
    acc = accuracies['train']
    val_acc = accuracies['val']
    loss = losses['train']
    val_loss = losses['val']
    epochs = range(1, len(acc) + 1)
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.figure()
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.show()

In [ ]:
# start training
model, accuracies, losses = train_model(model, criterion, optimizer)

In [ ]:
plot_train_history(accuracies, losses)

Clearly the model is overfitting, let's increase dropout rate from 30 to 70%:

In [ ]:
# instantiate model and optimizers
model2 = Model(0.7)
model2 = model2.to(device)
criterion2 = torch.nn.CrossEntropyLoss()
optimizer2 = optim.Adam(model2.parameters())

In [ ]:
# start training
model2, accuracies2, losses2 = train_model(model2, criterion2, optimizer2, num_epochs=50)

In [ ]:
plot_train_history(accuracies2, losses2)

The model is still overfitting, let's add L2 regularization to model's weights

In [ ]:
# instantiate model and optimizers
model3 = Model(0.7)
model3 = model3.to(device)
criterion3 = torch.nn.CrossEntropyLoss()
optimizer3 = optim.Adam(model3.parameters(), lr=1e-2, weight_decay=5e-4)

In [ ]:
model3, accuracies3, losses3 = train_model(model3, criterion3, optimizer3, num_epochs=50)

In [ ]:
plot_train_history(accuracies3, losses3)

In [ ]:
# instantiate model and optimizers
model4 = Model(0.7)
model4 = model4.to(device)
criterion4 = torch.nn.CrossEntropyLoss()
optimizer4 = optim.Adam(model4.parameters(), weight_decay=5e-4)

In [ ]:
model4, accuracies4, losses4 = train_model(model4, criterion4, optimizer4, num_epochs=50)

In [ ]:
plot_train_history(accuracies4, losses4)

#### 4.4 Exploring model predictions

Validation accuracy is varying between 53-55%. Let's explore the predictions by the model to see where it is doing well and where it is failing

In [ ]:
# an evaluation funtion

labels_ = [0, 1, 2, 3, 4, 5, 6]
target_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
def evaluate_model(model=model, data=dataloaders['val'], 
                   title='Confusion matrix', normalize=False,
                  labels_=labels_, target_names=target_names):
    y_true = []
    y_pred = []
    
    # first, get the predictions
    model.eval() # set model in evaluation mode
    with torch.no_grad():
        # Iterate over data.
        for inputs, labels in data:
            # inputs = inputs.to(device, dtype=torch.float)
            inputs = inputs.to(device, dtype=torch.float)
            y_true.append(labels.squeeze(1))
            labels = labels.view(labels.size()[0]).to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_pred.append(preds.to('cpu').numpy())
            
    # print classification report
    y_true, y_pred = np.concatenate(y_true), np.concatenate(y_pred)
    print(classification_report(y_true, y_pred, labels=labels_, target_names=target_names))
    
    # plot the confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
        
    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()


In [ ]:
evaluate_model()

In [ ]:
evaluate_model(model=model2)

In [ ]:
evaluate_model(model=model3)

In [ ]:
evaluate_model(model=model4)

In [ ]:
# Let's save the trained models
if not os.path.exists('/content/gdrive/My Drive/Face_detection/phase_1'):
    os.mkdir('/content/gdrive/My Drive/Face_detection/phase_1')

PATH_TMP = '/content/gdrive/My Drive/Face_detection/phase_1'
model_names = ['model', 'model2', 'model3', 'model4']
list_models = [model, model2, model3, model4]
for i in range(len(list_models)):
    torch.save(list_models[i].state_dict(), os.path.join(PATH_TMP, model_names[i]+'.pth'))


From the above evaluations, we can see that the worst classes in terms of recall are "Disgust" and "Fear" . Let's remove these 2 classes and see How the model will perform better.

In [ ]:
# remove Disgust and Fear classes
train_images_new = train_images[np.where((train_labels!=1) & (train_labels!=2))[0]]
train_labels_new = train_labels[np.where((train_labels!=1) & (train_labels!=2))[0]]

test_images_new = test_fer2013_images[np.where((test_fer2013_labels!=1) & (test_fer2013_labels!=2))[0]]
test_labels_new = test_fer2013_labels[np.where((test_fer2013_labels!=1) & (test_fer2013_labels!=2))[0]]

val_images_new = val_fer2013_images[np.where((val_fer2013_labels!=1) & (val_fer2013_labels!=2))[0]]
val_labels_new = val_fer2013_labels[np.where((val_fer2013_labels!=1) & (val_fer2013_labels!=2))[0]]

# Replace the other labels values to have a coherent 0 1 2, ... labelling
new_labels = {0: 0, 3: 1, 4: 2, 5: 3, 6: 4}
train_labels_new = np.array([new_labels[item] for item in train_labels_new])
test_labels_new = np.array([new_labels[item] for item in test_labels_new])
val_labels_new = np.array([new_labels[item] for item in val_labels_new])

In [ ]:
print(train_images_new.shape, train_labels_new.shape)
print(val_images_new.shape, val_labels_new.shape)
print(test_images_new.shape, test_labels_new.shape)

In [ ]:
24328/(3037+3006)

In [ ]:
np.unique(val_labels_new, return_counts=True)

In [ ]:
# Instanciate a transform object

# Create train, test and validation data objects
face_datasets_new = {
    'train': FaceData(train_images_new, train_labels_new, list_transforms),
    'val': FaceData(val_images_new, val_labels_new, list_transforms),
}

face_datasets_test_new = FaceData(test_images_new, test_labels_new, list_transforms)

# Data loaders
dataloaders_new = {
    phase: DataLoader(face_datasets_new[phase], batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    for phase in ['train', 'val']
}
dataset_sizes_new = {phase: len(face_datasets_new[phase]) for phase in ['train', 'val']}

dataloaders_test_new = DataLoader(face_datasets_test_new, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
dataset_sizes_test_new = len(face_datasets_test_new)

In [ ]:
# instanciate a new model with 5 classes
model5 = Model(0.7, 5)
model5 = model5.to(device)
criterion5 = torch.nn.CrossEntropyLoss()
optimizer5 = optim.Adam(model5.parameters(), weight_decay=5e-3)

In [ ]:
model5, accuracies5, losses5 = train_model(model5, criterion5, optimizer5, 
                                           data=dataloaders_new,
                                           dataset_sizes=dataset_sizes_new,
                                           num_epochs=50)

In [ ]:
plot_train_history(accuracies5, losses5)

In [ ]:
labels_ = [0, 1, 2, 3, 4]
target_names = ['Angry', 'Happy', 'Sad', 'Surprise', 'Neutral']
evaluate_model(model=model5, data=dataloaders_new['val'], labels_=labels_, target_names=target_names)

In [ ]:
PATH_TMP = '/content/gdrive/My Drive/Face_detection/phase_1'
torch.save(model5.state_dict(), os.path.join(PATH_TMP, 'model5.pth'))


### 5. Feature maps visualization

In [ ]:
class CamExtractor:
    
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
    
    def save_gradient(self, grad):
        self.gradients = grad
    
    def forward_pass_on_convolutions(self, x):
        conv_output = None
        for module_name, module in self.model._modules.items():
            print(module_name)
            if module_name == 'fc1':
                x = x.view(-1, x.size()[1])
            if module_name == 'fc3':
                return conv_output, x
            x = module(x) # forward pass
            if module_name == self.target_layer:
                print('OK')
                x.register_hook(self.save_gradient)
                conv_output = x
        
        return conv_output, x
    
    def forward_pass(self, x):
        conv_output, x = self.forward_pass_on_convolutions(x)
        # x = x.view(x.size(0), -1)
        x = self.model.fc3(x)
        return conv_output, x

class GradCam:
    
    def __init__(self, model, target_layer):
        self.model = model
        self.model.eval()
        self.extractor = CamExtractor(self.model, target_layer)
    
    def generate_cam(self, input_image, target_index=None):
        conv_output, model_output = self.extractor.forward_pass(input_image)
        if target_index == None:
            target_index = np.argmax(model_output.data.numpy())
        one_hot_output = torch.FloatTensor(1, model_output.size()[-1]).zero_().to(device)
        one_hot_output[0][target_index] = 1
        self.model.fc3.zero_grad()
        model_output.backward(gradient=one_hot_output, retain_graph=True)
        guided_gradients = self.extractor.gradients.data.cpu().numpy()[0]
        target = conv_output.data.cpu().numpy()[0]
        weights = np.mean(guided_gradients, axis=(1, 2))

        cam = np.ones(target.shape[1:], dtype=np.float32)
        for i, w in enumerate(weights):
            cam += w * target[i, :, :]
        print(cam.shape)
        cam = cv2.resize(cam, (48, 48))
        cam = np.maximum(cam, 0)
        cam = (cam - np.min(cam)) / (np.max(cam) - np.min(cam))
        cam = np.uint8(cam * 255)
        return cam
            
        
def save_class_activation_on_image(org_img, activation_map, file_name):
    cv2.imwrite(os.path.join(PATH_TMP, file_name+'_Cam_Grayscale.jpg'),
               activation_map)
    
    activation_heatmap = cv2.applyColorMap(activation_map, cv2.COLORMAP_JET)
    cv2.imwrite(os.path.join(PATH_TMP, file_name+'_Cam_Heatmap.jpg'),
               activation_heatmap)
    
    img_with_heatmap = np.float32(activation_heatmap) + np.float32(org_img)
    img_with_heatmap = img_with_heatmap / np.max(img_with_heatmap)
    cv2.imwrite(os.path.join(PATH_TMP, file_name+'_Cam_On_Image.jpg'),
               np.uint8(255 * img_with_heatmap))

def preprocess_image(np_image):
    image = np.expand_dims(np_image, 0)
    image = torch.from_numpy(image).float()
    image.unsqueeze_(0)
    return image

In [ ]:
def save_class_activation_on_image(org_img, activation_map, file_name):
    # cv2.imwrite(os.path.join(PATH_TMP, file_name+'_Cam_Grayscale.jpg'),
    #           activation_map)
    
    return cv2.applyColorMap(activation_map, cv2.COLORMAP_HSV)
    

In [ ]:
tmp_image = preprocess_image(fer2013_transformed[187])
tmp_image = tmp_image.to(device)

In [ ]:
grad_cam = GradCam(model5, target_layer='conv8')

In [ ]:
print(tmp_image.size())

In [ ]:
cam = grad_cam.generate_cam(tmp_image, 1)

In [ ]:
heatmap = save_class_activation_on_image(jaffe_transformed[10], cam, 'happy_1')

In [ ]:
plt.imshow(cam, cmap='gray')

In [ ]:
plt.imshow(heatmap)

In [ ]:
plt.imshow(fer2013_transformed[187], cmap='gray')

### 6. Using 224 x 224 image data

In [ ]:
%%time
fer2013_train = fer2013_array[np.where((fer2013_labels!=1) & (fer2013_labels!=2) & (flags=='Training'))[0]]
fer2013_test = fer2013_array[np.where((fer2013_labels!=1) & (fer2013_labels!=2) & (flags=='PrivateTest'))[0]]
fer2013_val = fer2013_array[np.where((fer2013_labels!=1) & (fer2013_labels!=2) & (flags=='PublicTest'))[0]]

new_labels = {0: 0, 3: 1, 4: 2, 5: 3, 6: 4}

fer2013_train_labels = np.array([new_labels[item] 
                                 for item in 
                                 fer2013_labels[np.where((fer2013_labels!=1) & (fer2013_labels!=2) & (flags=='Training'))[0]]])
fer2013_test_labels = np.array([new_labels[item] 
                                 for item in 
                                 fer2013_labels[np.where((fer2013_labels!=1) & (fer2013_labels!=2) & (flags=='PrivateTest'))[0]]])
fer2013_val_labels = np.array([new_labels[item] 
                                 for item in 
                                 fer2013_labels[np.where((fer2013_labels!=1) & (fer2013_labels!=2) & (flags=='PublicTest'))[0]]])

jaffe_train = jaffe_array[np.where((jaffe_labels!=1) & (jaffe_labels!=2))[0]]
jaffe_train_labels = np.array([new_labels[item] 
                                 for item in 
                                 jaffe_labels[np.where((jaffe_labels!=1) & (jaffe_labels!=2))[0]]])


In [ ]:
%%time
# get the data in dictionary 
training_data_dict = {}
training_data_jaffe_dict = {}
test_data_dict = {}
val_data_dict = {}

for i in range(5):
    training_data_dict[i] = fer2013_train[np.where(fer2013_train_labels==i)]
    training_data_jaffe_dict[i] = jaffe_train[np.where(jaffe_train_labels==i)]
    test_data_dict[i] = fer2013_test[np.where(fer2013_test_labels==i)]
    val_data_dict[i] = fer2013_val[np.where(fer2013_val_labels==i)]
    

In [ ]:
for i in range(5):
    print("------Class {} --------".format(i))
    print(training_data_dict[i].shape)
    print(training_data_jaffe_dict[i].shape)
    print(test_data_dict[i].shape)
    print(val_data_dict[i].shape)

In [ ]:
# download the pretrained face_detector file for dlib
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 -P /content/data
!bzip2 -dk /content/data/shape_predictor_68_face_landmarks.dat.bz2

In [ ]:
PATH_DETECTOR = '/content/data/shape_predictor_68_face_landmarks.dat'
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(PATH_DETECTOR)

In [ ]:
# Instanciate a face aligner object
fa = FaceAligner(predictor, desiredFaceWidth=256)

# function to process one image
def align_and_crop_one_new(im):

    gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
    # detect the face in the image
    rects = detector(gray, 2)
    if len(rects) > 0: # if we have at least one face detected
        rects = rects[0] # consider the 1st face
        faceAligned = fa.align(im, gray, rects) # align the face

        # cropping the image
        gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
        rects = detector(faceAligned, 2)
        if len(rects) > 0:
            rects = rects[0]
            
            (x, y, w, h) = rect_to_bb(rects) # get the bounding box coordinates
            
            faceCropped = transform.resize(faceAligned[y: y + h, 
                                                 x: x + w, 0], (224, 224))

            return (exposure.equalize_hist(faceCropped), 'OK1')
        else:
            # if no face is found in the aligned face, return just a resized aligned face 48x48
            return (exposure.equalize_hist(transform.resize(faceAligned[:, :, 0], (224, 224))), 'OK2')
    else:
        # if no face is found at all in the image, return the resized original image 48x48
        return (exposure.equalize_hist(transform.resize(im[:, :, 0], (224, 224))), 'OK3')


# function to align faces and compute landmarks
def align_and_crop_new(list_images, class_, name):
    
    # converting image to 8-bit
    print('--------------Converting images----------------')
    list_images = list(np.uint8(list_images))
    
    #resizing the  images
    print('--------------Resizing images----------------')
    list_images = [np.repeat(np.expand_dims(im, 2), 3, 2) 
                       for im in list_images]
    
    print('--------------Align and Crop----------------')
    with Pool() as p:
        results_tuple = p.map(align_and_crop_one_new, tqdm.tqdm(list_images))
    
    results1 = [item[0] for item in results_tuple]
    results2 = [item[1] for item in results_tuple]
    
    # apply histogram equalization to all images and return the result
    results1 = np.concatenate([np.expand_dims(item, 0) for item in results1])
    
    # save the processed images
    path = os.path.join('/content/data/', name+'_'+str(class_)+'.npy')
    np.save(path, results1)
    
    return dict(Counter(results2))


In [ ]:
%%time
counts_j = {}
for i in range(5):
    print('***********Processing class {}********'.format(i))
    counts_j[i] = align_and_crop_new(training_data_jaffe_dict[i], i, 'jaffe_train')

In [ ]:
%%time
counts_test = {}
for i in range(5):
    print('***********Processing class {}********'.format(i))
    counts_test[i] = align_and_crop_new(test_data_dict[i], i, 'fer2013_test')

In [ ]:
%%time
counts_val = {}
for i in range(5):
    print('***********Processing class {}********'.format(i))
    counts_val[i] = align_and_crop_new(val_data_dict[i], i, 'fer2013_val')
    

In [ ]:
%%time
!cp /content/data/*.npy /content/gdrive/My\ Drive/Face_detection/data_224

In [ ]:
%%time
counts_train = {}
for i in range(5):
    print('***********Processing class {}********'.format(i))
    counts_train[i] = align_and_crop_new(training_data_dict[i], i, 'fer2013_train')
    

In [ ]:
dirs = ['train', 'val', 'test']
classes = ['0', '1', '2', '3', '4']
p_drive = '/content/gdrive/My Drive/Face_detection/data_224/'
for d in dirs:
    for cl in classes:
        if not os.path.exists(os.path.join(p_drive, d)):
            os.mkdir(os.path.join(p_drive, d))
        if not os.path.exists(os.path.join(os.path.join(p_drive, d), cl)):
            os.mkdir(os.path.join(os.path.join(p_drive, d), cl))

In [ ]:
def save_as_image(paths):
    data = []
    for p in paths:
        data.append(np.load(p))
    cl = paths[0].split('_')[-1][:-4]
    d = paths[0].split('_')[-2]
    data = np.concatenate(data)
    
    for i, im in tqdm.tqdm(enumerate(data)):
        plt.imsave(arr=im, fname=os.path.join(p_drive, d, cl, str(i)+'.png'), cmap='gray')

In [ ]:
list_paths = sorted(glob.glob('/content/data/*.npy'))
list_paths_test_val = [[p] for p in list_paths if 'train' not in p]
list_paths_train = [[p1, p2] 
                    for p1, p2 in zip([p for p in list_paths if 'fer2013_train' in p],
                                     [p for p in list_paths if 'jaffe_train' in p])]
print(list_paths_train)
print(list_paths_test_val)

In [ ]:
for i, p in enumerate(list_paths_train):
    print('Saving class {}'.format(i))
    save_as_image(p)

In [ ]:
for p in list_paths_test_val:
    print('*******{}*******'.format(p))
    save_as_image(p)

In [ ]:
list_paths

In [ ]:
tmp2 = np.concatenate([tmp])

In [ ]:
tmp2 == tmp

In [ ]:
! du -ah /content/data/*train*

In [ ]:
1.5+2.75+1.9+1.2+1.9

In [ ]:
!ls /content/data

In [ ]:
print(counts_val)
print(counts_test)

In [ ]:
%%time
fer2013_transformed, counts = align_and_crop_new(fer2013_array)
jaffe_transformed, counts_j = align_and_crop_new(jaffe_array)

In [ ]:
fer

In [ ]:
tmp_image = np.repeat(np.expand_dims(fer2013_array[2], 2), 3, 2).astype(np.uint8)

In [ ]:
tmp_image2, ok = align_and_crop_one_new(tmp_image)

In [ ]:
plt.imshow(tmp_image, cmap='gray')

In [ ]:
plt.imshow(tmp_image2, cmap='gray')

In [ ]:
# loading data


###  5. Transfert learning 

#### 5.1 ResNet18

In [ ]:
# load the model
resnet18 = models.resnet18(pretrained=True)

In [ ]:
print(resnet18)

In [ ]:
class ModelTest(torch.nn.Module):
    
    def __init__(self, dropout=0.3):
        super(ModelTest, self).__init__()
        
        self.dropout = dropout
        
        # 1st block
        self.conv1 = SeparableConv(1, 64)
        self.batchnorm1 = torch.nn.BatchNorm2d(64)
        self.conv2 = SeparableConv(64, 64)
        self.batchnorm2 = torch.nn.BatchNorm2d(64)
        
        # 2nd block
        self.conv3 = SeparableConv(64, 128)
        self.batchnorm3 = torch.nn.BatchNorm2d(128)
        self.conv4 = SeparableConv(128, 128)
        self.batchnorm4 = torch.nn.BatchNorm2d(128)
        
        # 3rd block
        self.conv5 = SeparableConv(128, 256)
        self.batchnorm5 = torch.nn.BatchNorm2d(256)
        self.conv6 = SeparableConv(256, 256)
        self.batchnorm6 = torch.nn.BatchNorm2d(256)
        
        # 4th block
        self.conv7 = SeparableConv(256, 512)
        self.batchnorm7 = torch.nn.BatchNorm2d(512)
        self.conv8 = SeparableConv(512, 512)
        self.batchnorm8 = torch.nn.BatchNorm2d(512)
        
        self.avg_pool = torch.nn.AdaptiveAvgPool2d((1, 1))
        
        # 1st fc block
        self.fc1 = torch.nn.Linear(512, 7)

        
    def forward(self, x):
        #1st block
        x = self.conv1(x)
        x = F.relu(self.batchnorm1(x))
        x = self.conv2(x)
        x = F.relu(self.batchnorm2(x))
        x = F.max_pool2d(x, 2)
        
        # 2nd block
        x = self.conv3(x)
        x = F.relu(self.batchnorm3(x))
        x = self.conv4(x)
        x = F.relu(self.batchnorm4(x))
        x = F.max_pool2d(x, 2)
        
        # 3rd block
        x = self.conv5(x)
        x = F.relu(self.batchnorm5(x))
        x = self.conv6(x)
        x = F.relu(self.batchnorm6(x))
        # x = F.max_pool2d(x, 2)
                
        # 4th block
        x = self.conv7(x)
        x = F.relu(self.batchnorm7(x))
        x = self.conv8(x)
        x = F.relu(self.batchnorm8(x))
        # x = F.max_pool2d(x, 2)
        
        x = self.avg_pool(x)
        x = x.view(-1, x.size()[1])
        x = self.fc1(x)
        
        return x


In [ ]:
# instantiate model and optimizers
model5 = ModelTest()
model5 = model5.to(device)
criterion5 = torch.nn.CrossEntropyLoss()
optimizer5 = optim.Adam(model5.parameters())


In [ ]:
model5, accuracies5, losses5 = train_model(model5, criterion5, optimizer5, num_epochs=50)